In [1]:
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, execute, Aer
from qiskit.providers.aer import QasmSimulator
import qiskit_create_circ as qcc

In [ ]:
# dir(qcc)

In [16]:
N = 1
m = 1
n_i = 1
L = int(math.ceil(math.log(N + n_i, 2)))
print(L)
l = int(math.ceil(math.log(m + n_i, 2)) )
l2 = int(math.floor(math.log(m + n_i, 2)) + 1)
print(l)
print(l2)


pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg = qcc.allocateQubits(N, n_i, L)

# circuit = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)
circuit = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)

simulator = Aer.get_backend('statevector_simulator')

1
1
2
nqubits_a_b_phi:  1


In [ ]:
hReg

In [3]:
g_1, g_2, g_12 = 2, 1, 0
gp = math.sqrt(abs((g_1 - g_2) ** 2 + 4 * g_12 ** 2))
if g_1 > g_2:
    gp = -gp
g_a, g_b = (g_1 + g_2 - gp) / 2, (g_1 + g_2 + gp) / 2
u = math.sqrt(abs((gp + g_1 - g_2)/ (2 * gp)))
eps = .001

timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, Delta_phiList = [], [], [], [], [], [], []
qcc.populateParameterLists(N, timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, 
                       Delta_phiList, g_a, g_b, eps)


Test uCount

In [ ]:
# qcc.uCount(circuit, m, n_i, l, pReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)

# result = execute(circuit, simulator).result()
# statevector = result.get_statevector(circuit)

# print(statevector)

Test uE

In [4]:
qcc.uE(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg,
           Delta_phiList[0], Delta_aList[0], Delta_bList[0])

result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)

print(statevector)

counts list:  [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 2], [0, 1, 1], [0, 2, 0], [1, 0, 1], [1, 1, 0], [2, 0, 0]]
length counts list:  9
[-0.71065688+0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
  0.        +0.j  0.        +0.j]


In [5]:
print(statevector**2)

[0.5050332-0.j 0.       +0.j 0.       +0.j ... 0.       +0.j 0.       +0.j
 0.       +0.j]


In [6]:
circuit.draw()

»
 q0_0: ───────────────────────────────────────────────────────────»
                                                                  »
 q0_1: ───────────────────────────────────────────────────────────»
                                                                  »
 q0_2: ───────────────────────────────────────────────────────────»
                                                                  »
 q0_3: ───────────────────────────────────────────────────────────»
                                                                  »
 q0_4: ───────────────────────────────────────────────────────────»
                                                                  »
 q0_5: ───────────────────────────────────────────────────────────»
                                                                  »
 q1_0: ───────────────────────────────────────────────────────────»
                                                                  »
 q2_0: ───────────────────────────────────────────────────────────»
                      ┌────────────┐                              »
 q3_0: ───────────────┤ RY(1.4159) ├──────────────────────────────»
            ┌───┐     └─────┬──────┘     ┌───┐          ┌───┐     »
 q4_0: ─────┤ X ├──■────────┼─────────■──┤ X ├──────────┤ X ├──■──»
            └─┬─┘┌─┴─┐      │       ┌─┴─┐└─┬─┘          └─┬─┘┌─┴─┐»
 q4_1: ───────┼──┤ X ├──────■───────┤ X ├──┼──────────────┼──┤ X ├»
              │  └─┬─┘              └─┬─┘  │              │  └─┬─┘»
 q4_2: ───────┼────┼──────────────────┼────┼──────────────┼────┼──»
              │    │                  │    │              │    │  »
 q4_3: ───────┼────┼──────────────────┼────┼──────────────┼────┼──»
              │    │                  │    │              │    │  »
 q4_4: ───────┼────┼──────────────────┼────┼──────────────┼────┼──»
       ┌───┐  │    │                  │    │  ┌───┐       │    │  »
 q5_0: ┤ X ├──■────┼──────────────────┼────■──┤ X ├───────■────┼──»
       └───┘  │    │                  │    │  └───┘       │    │  »
 q6_0: ───────┼────┼──────────────────┼────┼──────────────┼────┼──»
              │    │                  │    │  ┌───┐       │    │  »
 q7_0: ───────┼────■──────────────────■────┼──┤ X ├───────┼────■──»
              │                            │  └───┘       │       »
 q8_0: ───────┼────────────────────────────┼──────────────┼───────»
       ┌───┐  │                            │  ┌───┐┌───┐  │       »
 q9_0: ┤ X ├──■────────────────────────────■──┤ X ├┤ X ├──■───────»
       └───┘                                  └───┘└───┘          »
q10_0: ───────────────────────────────────────────────────────────»
                                                                  »
«                                                                              »
« q0_0: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q0_1: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q0_2: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q0_3: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q0_4: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q0_5: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q1_0: ───────────────────────────────────────────────────────────────────────»
«                                                                              »
« q2_0: ───────────────────────────────────────────────────────

In [10]:
print(l)

1


In [7]:
circ = QuantumCircuit(pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)

In [8]:
qcc.numberControl(circ, l, 0, n_phiReg, w_phiReg)

Qubit(QuantumRegister(1, 'q9'), 0)

In [9]:
circ.draw()

q0_0: ─────
            
 q0_1: ─────
            
 q0_2: ─────
            
 q0_3: ─────
            
 q0_4: ─────
            
 q0_5: ─────
            
 q1_0: ─────
            
 q2_0: ─────
            
 q3_0: ─────
            
 q4_0: ─────
            
 q4_1: ─────
            
 q4_2: ─────
            
 q4_3: ─────
            
 q4_4: ─────
            
 q5_0: ─────
            
 q6_0: ─────
            
 q7_0: ─────
            
 q8_0: ─────
       ┌───┐
 q9_0: ┤ X ├
       └───┘
q10_0: ─────

Test U_h

In [ ]:
print(wReg[4])

In [13]:
# print(P_bList)
qcc.U_h(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, pReg, hReg, w_hReg,
            P_phiList[0], P_aList[0], P_bList[0])

IndexError: list index out of range

In [14]:
def reverse(lst):
    """reverse a list in place"""
    lst.reverse()
    return lst

def intToBinary(l, number):
    """Converts integer to binary list of size l with LSB first and MSB last"""
    numberBinary = [int(x) for x in list('{0:0b}'.format(number))]
    numberBinary = (l - len(numberBinary)) * [0] + numberBinary
    return reverse(numberBinary)

def generateGrayList(l, number):
    """
    l is the size of the current count register
    Return list of elements in gray code from |0> to |number> where each entry is of type[int, binary list].
    int: which bit is the target in the current iteration, binary list: the state of the rest of the qubits (controls)
    """
    grayList = [[0, l * [0]]]
    targetBinary = intToBinary(l, number)
    for index in range(len(targetBinary)):
        if targetBinary[index] == 1:
            grayList.append([index, (list(grayList[-1][1]))])
            print("gray list: ", grayList[-1])
            grayList[-1][1][index] = 1
    return grayList[1:]

In [15]:
generateGrayList(1, 2)

gray list:  [1, [0]]


IndexError: list assignment index out of range

Testing U_p

In [ ]:
qcc.U_p(circuit, l, n_i, 0, pReg, hReg, w_hReg, wReg, g_a, g_b)

result = execute(circuit, simulator).result()
statevector = result.get_statevector(circuit)

print(statevector)